# Explore here

In [41]:
import pandas as pd

In [42]:
total_data = pd.read_csv("/Users/jesus/Desktop/4geeks/4geeks - naive bayes/jesus-cuenca-naive-bayes-machine-learning-python-template/data/raw/playstore_reviews.csv")

total_data.head()

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0


In [43]:
import re
import string
# Función para limpiar el texto
def limpiar_texto(texto):
    # Eliminar emoticonos (esto es solo un ejemplo, dependiendo del tipo de emoticonos puedes necesitar más reglas)
    texto = re.sub(r'[^\w\s]', '', texto)  # Elimina todo lo que no sea palabras o espacios
    
    # Eliminar todos los signos de exclamación y puntuación
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    
    # Eliminar cualquier emoticono (ejemplo: :) o :D o cualquier combinación)
    texto = re.sub(r'[:;][\-]?[)D]', '', texto)  # Elimina algunos emoticonos comunes
    
    # Eliminar números si solo quieres palabras (opcional)
    texto = re.sub(r'\d+', '', texto)  # Elimina los números
    
    #Convertir a minúsculas para normalizar
    texto = texto.lower()
    
    return texto

# Aplicar la función a la columna 'review'
total_data['review'] = total_data['review'].apply(limpiar_texto)

In [44]:
total_data["review"]

0       privacy at least put some option appear offli...
1       messenger issues ever since the last update i...
2       profile any time my wife or anybody has more ...
3       the new features suck for those of us who don...
4       forced reload on uploading pic on replying co...
                             ...                        
886     loved it i loooooooooooooovvved it because it...
887     all time legendary game the birthday party le...
888     ads are way to heavy listen to the bad review...
889     fun works perfectly well ads arent as annoyin...
890     theyre everywhere i see angry birds everywher...
Name: review, Length: 891, dtype: object

In [45]:
from sklearn.model_selection import train_test_split

X = total_data["review"]  # Las características (texto de los comentarios)
y = total_data["polarity"]  # La variable objetivo (0 = negativo, 1 = positivo)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [46]:
from sklearn.feature_extraction.text import CountVectorizer

vec_model = CountVectorizer(stop_words = "english")
X_train = vec_model.fit_transform(X_train).toarray()
X_test = vec_model.transform(X_test).toarray()

In [47]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

hyperparams = {
    "alpha": [0.01, 0.1, 0.5, 1.0, 2.0, 5.0],  # Control de suavizado
    "force_alpha": [True, False],  # Si se debe forzar alpha
    "binarize": [None, 0.0, 0.5, 1.0],  # Umbral de binarización para variables continuas
    "fit_prior": [True, False],  # Si se debe aprender una probabilidad a priori
    "class_prior": [None, [0.2, 0.8], [0.5, 0.5], [0.7, 0.3]]  # Probabilidad previa de clases (None = automático)
}

# Crear el modelo
model = BernoulliNB()

# Configurar GridSearchCV
grid = GridSearchCV(model, hyperparams, scoring="accuracy", cv=5, n_jobs=-1)


In [48]:
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=BernoulliNB(), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.1, 0.5, 1.0, 2.0, 5.0],
                         'binarize': [None, 0.0, 0.5, 1.0],
                         'class_prior': [None, [0.2, 0.8], [0.5, 0.5],
                                         [0.7, 0.3]],
                         'fit_prior': [True, False],
                         'force_alpha': [True, False]},
             scoring='accuracy')

In [49]:
grid

GridSearchCV(cv=5, estimator=BernoulliNB(), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.1, 0.5, 1.0, 2.0, 5.0],
                         'binarize': [None, 0.0, 0.5, 1.0],
                         'class_prior': [None, [0.2, 0.8], [0.5, 0.5],
                                         [0.7, 0.3]],
                         'fit_prior': [True, False],
                         'force_alpha': [True, False]},
             scoring='accuracy')

In [50]:
grid.best_estimator_

BernoulliNB(alpha=0.01, binarize=None)

In [51]:
# Obtener el mejor modelo
best_model = grid.best_estimator_

# Hacer predicciones
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

# Calcular accuracy
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Imprimir resultados
print("Accuracy en train:", train_accuracy)
print("Accuracy en test:", test_accuracy)


Accuracy en train: 0.9915730337078652
Accuracy en test: 0.8268156424581006


bagofwords

In [53]:
from sklearn.feature_extraction.text import CountVectorizer

# Verificar la longitud de la columna "review"
print(f"Número de filas en 'review': {len(total_data['review'])}")

# Inicializar el vectorizador
vectorizer = CountVectorizer()

# Ajustar y transformar los datos de la columna "review"
X_bag_of_words = vectorizer.fit_transform(total_data["review"])

# Mostrar el tamaño de la matriz resultante
print(f"Forma de la matriz Bag of Words: {X_bag_of_words.shape}")

# Obtener el vocabulario generado
vocabulario = vectorizer.vocabulary_
print(f"Vocabulario generado: {list(vocabulario.items())[:10]}")  # Muestra las primeras 10 palabras

# Convertir la matriz dispersa a un array denso (opcional, puede ser costoso en memoria)
X_array = X_bag_of_words.toarray()

# Obtener las palabras más frecuentes
feature_names = vectorizer.get_feature_names_out()
print(f"Palabras más frecuentes: {feature_names[:10]}")  # Muestra las primeras 10 palabras

# Inversa: Obtener el texto original desde Bag of Words
texto_original = vectorizer.inverse_transform(X_bag_of_words)
print(f"Ejemplo de texto transformado inversamente: {texto_original[:5]}")  # Muestra los primeros 5



Número de filas en 'review': 891
Forma de la matriz Bag of Words: (891, 4189)
Vocabulario generado: [('privacy', 2713), ('at', 245), ('least', 1956), ('put', 2774), ('some', 3270), ('option', 2478), ('appear', 186), ('offline', 2427), ('mean', 2151), ('for', 1344)]
Palabras más frecuentes: ['aa' 'aafnaii' 'aakhirat' 'aalikati' 'aap' 'aaps' 'aapsssssss' 'aaru'
 'abilities' 'ability']
Ejemplo de texto transformado inversamente: [array(['privacy', 'at', 'least', 'put', 'some', 'option', 'appear',
       'offline', 'mean', 'for', 'people', 'like', 'me', 'its', 'big',
       'pressure', 'to', 'be', 'seen', 'online', 'you', 'need',
       'response', 'on', 'every', 'message', 'or', 'else', 'called',
       'seenzone', 'only', 'if', 'wanna', 'do', 'facebook', 'is', 'read',
       'my', 'newsfeed', 'and', 'just', 'want', 'pls', 'reconsidered',
       'review', 'tried', 'turn', 'off', 'chat', 'but', 'still', 'can',
       'see', 'as'], dtype='<U24'), array(['to', 'you', 'message', 'or', 'facebo

tf-idf Encoding

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Configurar la precisión en la impresión de numpy
np.set_printoptions(precision=2)

# Inicializar el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Ajustar y transformar los datos de la columna "review"
X_tfidf = tfidf_vectorizer.fit_transform(total_data["review"])

# Convertir la matriz TF-IDF a un array denso
X_tfidf_array = X_tfidf.toarray()

# Imprimir la matriz TF-IDF con valores redondeados
print(X_tfidf_array)

# Obtener las palabras más frecuentes en el vocabulario
feature_names = tfidf_vectorizer.get_feature_names_out()
print(f"Palabras más frecuentes: {feature_names[:10]}")  # Muestra las primeras 10 palabras


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Palabras más frecuentes: ['aa' 'aafnaii' 'aakhirat' 'aalikati' 'aap' 'aaps' 'aapsssssss' 'aaru'
 'abilities' 'ability']


In [58]:
import pickle

# Guardar el GridSearchCV completo
with open("/Users/jesus/Desktop/4geeks/4geeks - naive bayes/jesus-cuenca-naive-bayes-machine-learning-python-template/data/processed/diccionarios/grid_search.pkl", "wb") as f:
    pickle.dump(grid, f)

# Guardar el mejor modelo entrenado
with open("/Users/jesus/Desktop/4geeks/4geeks - naive bayes/jesus-cuenca-naive-bayes-machine-learning-python-template/data/processed/diccionarios/best_model.pkl", "wb") as f:
    pickle.dump(best_model, f)

#Guardar la vectorización
with open("/Users/jesus/Desktop/4geeks/4geeks - naive bayes/jesus-cuenca-naive-bayes-machine-learning-python-template/data/processed/diccionarios/vec_model.pkl", "wb") as f:
    pickle.dump(vec_model, f)